<a href="https://colab.research.google.com/github/insaneonai/Fromscratchdl/blob/main/Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Implementation of Resnet using Pytorch 🤗

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.7,0.7,0.7),(0.7,0.7,0.7))  ## Normalize to Mean and standard deviation.
])

In [ ]:

batch_size = 8

train_set = torchvision.datasets.CIFAR10(root="/content/",train=True,transform=transforms,download=True)
train_loader = torch.utils.data.DataLoader(dataset=train_set,batch_size=batch_size,shuffle=True)

100%|██████████| 170498071/170498071 [00:10<00:00, 15724388.23it/s]


Extracting /content/cifar-10-python.tar.gz to /content/


In [ ]:
test_set = torchvision.datasets.CIFAR10(root="/content/",train=False,transform=transforms,download=True)
test_loader = torch.utils.data.DataLoader(dataset=test_set,batch_size=batch_size,shuffle=False)

Files already downloaded and verified


In [ ]:
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
class Block(nn.Module):
  def __init__(self,in_channels,out_channels,stride=1,downsampler=None):
    super().__init__()
    self.expansion = 4
    self.conv1 = nn.Conv2d(in_channels,out_channels,1,1,0,bias=False)
    self.bn1 = nn.BatchNorm2d(out_channels)
    self.conv2 = nn.Conv2d(out_channels,out_channels,3,stride,padding=1,bias=False)
    self.bn2 = nn.BatchNorm2d(out_channels)
    self.conv3 = nn.Conv2d(out_channels,out_channels*self.expansion,1,1,padding=0,bias=False)
    self.bn3 = nn.BatchNorm2d(out_channels*self.expansion)
    self.gelu = nn.GELU()
    self.downsampler = downsampler

  def forward(self,x):
    clone = x.clone()
    x = self.conv1(x)
    x = self.bn1(x)
    x = self.gelu(x)
    x = self.conv2(x)
    x = self.bn2(x)
    x = self.gelu(x)
    x = self.conv3(x)
    x = self.bn3(x)

    if self.downsampler:
      print("Using Downsampler")
      clone = self.downsampler(clone)
    print("clone shape: ", clone.shape, "x shape: ",x.shape)
    x += clone
    x = self.gelu(x)
    return x

In [ ]:
class RESNet(nn.Module):
  def __init__(self,block,img_channels,num_blocks_per_layer,out_class):
    super().__init__()
    self.in_channels = 64
    self.conv = nn.Conv2d(img_channels,64,kernel_size=7,stride=2,padding=3,bias=False)
    self.bn = nn.BatchNorm2d(64)
    self.gelu = nn.GELU()
    self.maxpool = nn.MaxPool2d(3,stride=2,padding=1)

    self.layer1 = self.create_layer(block,num_blocks_per_layer[0],64,1)
    self.layer2 = self.create_layer(block,num_blocks_per_layer[1],64*2,2)
    self.layer3 = self.create_layer(block,num_blocks_per_layer[2],64*3,2)
    self.layer4 = self.create_layer(block,num_blocks_per_layer[3],64*4,2)
    self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
    self.drouput = torch.nn.Dropout(p=0.5, inplace=False)
    self.fc = nn.Linear(512*2,out_class)
  def forward(self,x):
    x = self.conv(x)
    x = self.bn(x)
    x = self.gelu(x)
    x = self.maxpool(x)
    x = self.layer1(x)
    x = self.drouput(x)
    x = self.layer2(x)
    x = self.drouput(x)
    x = self.layer3(x)
    x = self.drouput(x)
    x = self.layer4(x)
    x = self.avgpool(x)
    x = x.reshape(x.shape[0], -1)
    x = self.fc(x)
    return x
  def create_layer(self,block,num_blocks,channels,stride):
    ## Must check condition to add x with F(x).
    downsample = None
    layers = []
    if stride !=1 or self.in_channels != channels * 4:
      downsample = nn.Sequential(nn.Conv2d(self.in_channels,channels * 4, 1, stride),
                                 nn.BatchNorm2d(channels * 4))

    layers.append(block(self.in_channels,channels,stride,downsample))
    self.in_channels = channels * 4
    for i in range(num_blocks-1):
      layers.append(block(self.in_channels,channels))
    return nn.Sequential(*layers)

In [ ]:
def ResNet50(img_channel=3, num_classes=10):
    return RESNet(Block, img_channel, [3, 4, 6, 3], num_classes)

In [ ]:
resnet = ResNet50()

In [ ]:
next(iter(train_set))[0].reshape(1,3,32,32)

In [ ]:
a = next(iter(train_set))[0]
b = next(iter(train_set))[0]

In [ ]:
def test():
    net = ResNet50(img_channel=3, num_classes=10)
    device = "cuda" if torch.cuda.is_available() else "cpu"
    y = net(torch.randn(4, 3, 224, 224)).to(device)
    print(y.size())

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()